# Market-maker results
This notebook runs a short demo of the Avellaneda-like market maker, loads the generated equity series, and plots: equity curve, drawdown, and rolling Sharpe.

Run the demo cell below to produce `notebooks/summary_metrics_demo.csv` and per-run equity CSVs in `notebooks/` (if available).

In [ ]:
# Run the demo runner (this will create notebooks/summary_metrics_demo.csv)
import subprocess
import sys
print('Running demo...')
ret = subprocess.run([sys.executable, 'tools/demo_market_maker.py'], capture_output=True, text=True)
print(ret.stdout)
if ret.stderr:
    print('Errors:', ret.stderr)

In [ ]:
# Load equity CSV produced by the demo and plot using qt.analytics.metrics helpers
import csv, os
from qt.analytics.metrics import plot_equity_curve, plot_drawdown, plot_rolling_sharpe
import matplotlib.pyplot as plt

demo_csv = 'notebooks/summary_metrics_demo.csv'
# If a per-run equity CSV exists, prefer that (files named starting with equity_)
equity_file = None
for fn in os.listdir('notebooks') if os.path.exists('notebooks') else []:
    if fn.startswith('equity_') and fn.endswith('.csv'):
        equity_file = os.path.join('notebooks', fn)
        break

if equity_file and os.path.exists(equity_file):
    ts = []
    vals = []
    with open(equity_file, 'r') as f:
        r = csv.reader(f)
        next(r, None)
        for row in r:
            ts.append(float(row[0]))
            vals.append(float(row[1]))
else:
    # fallback: try to read summary_metrics_demo.csv (which contains key metrics)
    vals = None

if vals is None:
    print('No per-run equity CSV found in notebooks/. Run the demo cell above to generate it.')
else:
    ax = plot_equity_curve(vals, title='Demo Equity Curve')
    plt.show()
    ax2 = plot_drawdown(vals, title='Demo Drawdown')
    plt.show()
    ax3 = plot_rolling_sharpe(vals, window=20)
    plt.show()